In [1]:
import datafaucet as dfc

Datafaucet is a productivity framework for ETL, ML application. Simplifying some of the common activities which are typical in Data pipeline such as project scaffolding, data ingesting, start schema generation, forecasting etc.

## Loading and Saving Parquet Data

In [ ]:
dfc.project.load('minimal')

 [datafaucet] NOTICE parquet.ipynb:engine:__init__ | Connecting to spark master: local[*]


In [ ]:
dfc.metadata.profile()

### Filter and projections Filters push down on parquet files

The following show how to selectively read files on parquet files (with partitions)

#### Create data

In [ ]:
df = dfc.range(10000).cols.create('g').randchoice([0,1,2,3])
df.cols.groupby('g').agg('count').data.grid()

#### Save data as parquet objects

In [ ]:
df.repartition('g').save('local', 'groups.parquet');

In [ ]:
dfc.list('data/save/groups.parquet').data.grid()

#### Read data parquet objects (with pushdown filters)

In [ ]:
spark = dfc.engine().context

In [ ]:
df = spark.read.load('data/save/groups.parquet')

In [ ]:
### No pushdown on the physical plan
df.explain()

In [ ]:
### Pushdown only column selection
df.groupby('g').count().explain()

In [ ]:
# push down row filter only but take all partitions
df.filter('id>100').explain()

In [ ]:
# pushdown partition filters and row (columnar) filters
df.filter('id>100 and g=1').groupby('g').count().explain()